# Tutorial: building quantum programs with Qililab. 






### What is pulse-level programming?

Unlike programming quantum circuits, which are transpiled into pulse schedules before execution, pulse-level programming allows direct control over the pulses that are being sent to the quantum computer. This low-level programming approach bypasses transpilation, avoiding inefficiencies and enabling precise control over which pulse is sent to which bus at what time.

Qililab supports pulse-level programming of quantum computers through QProgram: a hardware-agnostic pulse-level programming interface for describing quantum programs. 


<div>
<img src="media/pulses_1.gif" width="500"/>
</div>


### Pulse types

We can categorize pulses into three types based on their purpose:



<span style="color:darkblue"> **Drive pulses.** </span> These are microwave (MW) pulses applied to the qubit at its transition frequency. Drive pulses are used to manipulate the quantum state of the qubit, allowing it to move to any point on the Bloch sphere.

 <span style="color:darkblue"> **Readout pulses.**</span> These MW pulses are sent through a feedline to the resonators coupled with the qubits, enabling us to read the qubit's state. The state information is carried by the pulse out of the fridge and then digitized.

 <span style="color:darkblue"> **Flux pulses.** </span> These are voltage pulses used to adjust the trapped flux in the superconducting loop, altering the qubit's energy spectrum. Flux pulses are essential for executing two-qubit gates by aligning the qubit spectra to bring specific transitions into resonance, enabling the required interaction.

### Pulse waveforms 

With pulse-level programming, we have control over the shape of the pulses (waveforms) used to build a QProgram. Qililab helps us do so with the following classes:

For each microwave (MW) pulse, we define its in-phase (I) and quadrature (Q) components using the <span style="color:darkblue"> **IQPair**  </span> class. To use only the in-phase component, simply set the quadrature component to an array of zeros. Qililab supports a variety of waveforms, such as:

<span style="color:darkblue"> **Square**. </span> Defines a square waveform of certain amplitude and duration.

<span style="color:darkblue"> **Gaussian**.</span> Define a gaussian waveform of certain amplitude, duration and standard deviation.

<span style="color:darkblue"> **Arbitrary**.  </span> Creates waveforms of arbitrary shape, defining the amplitude of the pulse point by point with an array.

In [ ]:
# Example: creating the waveforms for a readout and a drive pulse

from qililab import IQPair, Gaussian, Square, Arbitrary
import numpy as np

# Define the duration and amplitude for the waveforms
drive_duration = 100  # in ns
drive_amplitude = 0.5  # in voltage, normalized units
readout_duration = 200  # in ns
readout_amplitude = 0.7  # in voltage, normalized units

drive_wf = IQPair( I=Gaussian( amplitude=drive_amplitude, duration=drive_duration, num_sigmas=4.0 ), Q=Arbitrary(np.zeros(drive_duration)))
readout_wf = IQPair(I=Square(amplitude=readout_amplitude, duration=readout_duration), Q=Square(amplitude=readout_amplitude, duration=readout_duration))

### Building a Qprorgam

The goal of a quantum program is to execute a synchronized sequence of pulses targeting different elements of the quantum chip. QProgram helps us achieve this with a variety of methods. The most important ones are:

- <span style="color:darkblue"> **Playing the Pulses**. </span> To play a pulse, we use the `play()` method, providing the pulse waveform and the bus it is directed to as arguments.

- <span style="color:darkblue">   **Synchronizing Pulses**. </span>  We can control the timing of pulses by adding wait times to specific buses using wait(). To synchronize pulses across multiple buses, we use the `sync()` method.

- <span style="color:darkblue"> **Data Acquisition** . </span>   After sending the measurement pulse, we use `acquire()` to trigger the digitalization of the incoming signal.

- <span style="color:darkblue"> **For Loops**. </span>  To loop over a sequence while varying a parameter (e.g., one of the pulses' amplitude or frequency), we use `for_loop()`. The variable must be defined first, specifying its domain.

- <span style="color:darkblue"> **Averaging Loops**. </span>  The hw_avg() method repeats the sequence multiple times and perfroms an average, which is crucial obtaining a clear signal.

Let's take a closer look at the syntax with an example:

In [ ]:
import qililab as ql
from qililab import QProgram

# initialise qprogram class
my_qp = QProgram()

# define the variable(s) to loop over
shots= my_qp.variable(domain=ql.Domain.Scalar)

# define weight for the aquisition
weights= IQPair(I=Square(amplitude=1.0, duration=readout_duration), Q=Square(amplitude=1.0, duration=readout_duration))
                          
# define sequence 
with my_qp.for_loop(variable=shots, start=1, stop=1000, step=1):

    # play drive pulse
    my_qp.play(waveform= drive_wf, bus= "drive_bus")
    
    # sync the drive and readout buses
    my_qp.sync()

    # play measurement pulse and data acquisition
    my_qp.play(waveform= readout_wf, bus= "readout_bus")
    my_qp.wait(duration=200, bus= "readout_bus") # add a wait before the aqusition, to compensate for the time of flight: te time it takes to the pulse travel through the line
    my_qp.acquire(weights=weights, bus= "readout")

### Retrieving information from the runcard

The runcard contains all necessary information about our laboratory setup (instruments, connection mappings, ips, etc) as well as the calibrated parameters for the quantum chip being used (pulse amplitudes, distorsions, pulse offsets, etc). 

Checking the information contained in it might help us write a better qprogram for the experiment we are tryng to perform. For instance, you can verify the names of the buses, the number of qubits, and their connectivity. Additionally, you can utilize the optimized gate parameters outlined in the runcard to create your pulses.

In [ ]:
# Retrieve the runcard of the current system

from qiboconnection.api import API
from qiboconnection.connection import ConnectionConfiguration

api = API(ConnectionConfiguration(username="youruser", api_key="yourkey"))
api.select_device_id(16)

# Retrieve runcard
runcard = api.get_runcard(runcard_name="saruman")
runcard_as_dict = runcard.runcard

# check buses
runcard_as_dict["buses"]

# check gates
runcard_as_dict["gates_settings"]["gates"]